In [1]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.4/904.4 kB 31.5 MB/s eta 0:00:00


In [ ]:
!unzip -q /home/input/image-preprocessed/_output_.zip

In [ ]:
import os
import pandas as pd
from PIL import Image
data_dir = '/home/input/cytol-gia-data/'
filepaths = []
folds = os.listdir(data_dir)
for fold in folds:
    foldpath = os.path.join(data_dir, fold)
    filelist = os.listdir(foldpath)
    if fold in ['ig', 'neutrophil']:
        continue
    for file in filelist:
        fpath = os.path.join(foldpath, file)
        
        filepaths.append(fpath)
       

# Concatenate data paths with labels into one dataframe
Fseries = pd.Series(filepaths, name= 'filepaths')

dff = pd.concat([Fseries], axis= 1)

# Extract the file name from the path
dff['NAME'] = dff['filepaths'].apply(lambda x: x.split('/')[-1])
train = pd.read_csv("/home/input/csv-data-clyclo/cytologia-data-1732098640162.csv")
dfcly = pd.merge(left = dff , right = train, on = 'NAME', how = 'inner')
# # Assuming 'df' is your pandas DataFrame and 'class' is the column name

# # Extract unique class labels
# labels = df['class'].unique().tolist()

# # Create a dictionary to map labels to integer IDs
# label_to_id = {label: i for i, label in enumerate(labels)}

# # Map the labels to integer IDs
# df['class'] = df['class'].map(label_to_id)



In [4]:
# Function to get image size
def get_image_size(filepath):
    try:
        with Image.open(filepath) as img:
            return f"{img.width}x{img.height}"
    except FileNotFoundError:
        return "File not found"
    except Exception as e:
        return str(e)

# Apply the function to the filepaths column
dfcly["image_size"] = dfcly["filepaths"].apply(get_image_size)

In [ ]:
dfcly.head()

In [ ]:
import os
import pandas as pd

# Define the path to the main folder
main_folder = "/home/working/train_valid"

# Initialize an empty list to store the data
data = []

# Walk through each subfolder in train, valid, and test
for split in ["train", "val", "test"]:
    split_folder = os.path.join(main_folder, split)
    
    # Ensure the split folder exists
    if os.path.exists(split_folder):
        for class_name in os.listdir(split_folder):
            class_folder = os.path.join(split_folder, class_name)
            
            # Ensure it's a directory
            if os.path.isdir(class_folder):
                for image_name in os.listdir(class_folder):
                    image_path = os.path.join(class_folder, image_name)
                    
                    # Add to the data list if it's a file
                    if os.path.isfile(image_path):
                        # Split the image name into base name and augmentations
                        parts = image_name.split("_", 1)
                        
                        if len(parts) == 2:  # If there's at least one underscore
                            base_name = parts[0] + ".jpg"
                            augmented = parts[1].rsplit(".", 1)[0]  # Remove file extension from augmentation
                            augmented_path = os.path.join(class_folder, image_name)
                        else:  # Handle cases with no underscore
                            base_name = image_name
                            augmented = None
                            augmented_path = image_path
                        
                        # Append the data with file paths
                        data.append({
                            "split": split,
                            "class": class_name,
                            "NAME": base_name,
                            "augmented": augmented,
                            "filepathsaug": image_path,        # Original image path
                     
                        })

# Convert the list to a DataFrame
df = pd.DataFrame(data)



In [7]:
# Check if the specific value exists in the NAME column
exists = '6a065480-9_fliped.jpg' in df['NAME'].values
print("Exists:", exists)


Exists: False


In [ ]:
merged_df = pd.merge(df,dfcly, on="NAME", how="left")
merged_df.head()

In [9]:
merged_df[merged_df['image_size'].isna()]

,split,class_x,NAME,augmented,filepathsaug,filepaths,x1,y1,x2,y2,class_y,image_size


In [10]:
# Function to apply augmentations
def adjust_coordinates(row):
    # Parse image size
    try:
        width, height = map(int, row["image_size"].split("x"))
    except ValueError:
        return row["x1"], row["y1"], row["x2"], row["y2"]  # Return original if size is invalid
    
    x1, y1, x2, y2 = row["x1"], row["y1"], row["x2"], row["y2"]

    if row["augmented"] == "fliped_h":  # Horizontal flip
        new_x1 = width - x2
        new_x2 = width - x1
        return new_x1, y1, new_x2, y2
    elif row["augmented"] == "fliped_v":  # Vertical flip
        new_y1 = height - y2
        new_y2 = height - y1
        return x1, new_y1, x2, new_y2
    elif row["augmented"] == "fliped_xy":  # Both horizontal and vertical flip
        new_x1 = width - x2
        new_x2 = width - x1
        new_y1 = height - y2
        new_y2 = height - y1
        return new_x1, new_y1, new_x2, new_y2
    else:  # No change for other cases
        return x1, y1, x2, y2

# Apply the transformation
merged_df[["new_x1", "new_y1", "new_x2", "new_y2"]] = merged_df.apply(adjust_coordinates, axis=1, result_type="expand")


In [11]:
finaldf = merged_df[['split', 'NAME','filepathsaug','new_x1', 'new_y1','new_x2', 'new_y2', 'class_x', 'image_size','augmented']]

In [12]:
finaldf.columns = ['split', 'NAME','filepaths','x1', 'y1','x2', 'y2', 'class', 'image_size','augmented']

In [ ]:
finaldf.head()

In [14]:
train_df = finaldf[finaldf['split'] == 'train']
val_df = finaldf[finaldf['split'] != 'train']

In [ ]:
import os
import pandas as pd
from PIL import Image

# Define class mapping
class_mapping = {name: idx for idx, name in enumerate(train_df["class"].unique())}

# Output directories
output_images_dir = "/home/working/dataset_new/train/images"
output_labels_dir = "/home/working/dataset_new/train/labels"
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

# Create YOLO-compatible annotations
for _, row in train_df.iterrows():
    # Load the image to get dimensions
    image_path = row["filepaths"]
    image = Image.open(image_path)
    image_width, image_height = image.size

    # Calculate YOLO format
    x_center = ((row["x1"] + row["x2"]) / 2) / image_width
    y_center = ((row["y1"] + row["y2"]) / 2) / image_height
    width = (row["x2"] - row["x1"]) / image_width
    height = (row["y2"] - row["y1"]) / image_height
    class_id = class_mapping[row["class"]]

    # Save annotation
    label_path = os.path.join(output_labels_dir, f"{os.path.basename(row['filepaths']).split('.')[0]}.txt")
    with open(label_path, "w") as f:
        f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    # Copy image to output directory
    image.save(os.path.join(output_images_dir, os.path.basename(row["filepaths"])))

# Save class names
with open("/home/working/dataset_new/classes.txt", "w") as f:
    for class_name in class_mapping:
        f.write(f"{class_name}\n")

print("Dataset train preparation complete!")

Dataset train preparation complete!


In [ ]:
import os
import pandas as pd
from PIL import Image


# Output directories
output_images_dir = "/home/working/dataset_new/val/images"
output_labels_dir = "/home/working/dataset_new/val/labels"
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

# Create YOLO-compatible annotations
for _, row in val_df.iterrows():
    # Load the image to get dimensions
    image_path = row["filepaths"]
    image = Image.open(image_path)
    image_width, image_height = image.size

    # Calculate YOLO format
    x_center = ((row["x1"] + row["x2"]) / 2) / image_width
    y_center = ((row["y1"] + row["y2"]) / 2) / image_height
    width = (row["x2"] - row["x1"]) / image_width
    height = (row["y2"] - row["y1"]) / image_height
    class_id = class_mapping[row["class"]]

    # Save annotation
    label_path = os.path.join(output_labels_dir, f"{os.path.basename(row['filepaths']).split('.')[0]}.txt")
    with open(label_path, "w") as f:
        f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    # Copy image to output directory
    image.save(os.path.join(output_images_dir, os.path.basename(row["filepaths"])))

print("Dataset val preparation complete!")

Dataset val preparation complete!


In [17]:
# !pip install -q split-folders


In [ ]:
# import splitfolders

# # Define input and output directories
# input_folder = '/home/working/dataset'
# output_folder = '/home/working/dataset_split'

# # Split with a ratio
# splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(0.95, 0.05), group_prefix=None, move=False)


In [ ]:
# import os
# import glob
# labels_dir = '/home/working/dataset_split/val/labels'  # e.g., '/home/working/datasets/dataset_split/train/labels'
# # Use glob to find all .txt files in the specified directory
# txt_files = glob.glob(os.path.join(labels_dir, '*.txt'))

# # Iterate over each file and print its contents
# for file_path in txt_files:
#     with open(file_path, 'r') as file:
#         print(f'Contents of {file_path}:')
#         for line in file:
#             print(line.strip())
#         print('-' * 40)  # Separator between files


In [ ]:
# Define the path to the class.txt file
class_file_path = "/home/working/dataset_new/classes.txt"

# Read the contents of the class.txt file
with open(class_file_path, "r") as file:
    class_list = [line.strip() for line in file.readlines()]



In [ ]:
# Define the path to the classes.txt file
class_file_path = "/home/working/dataset_new/classes.txt"

# Read the contents of the class.txt file
with open(class_file_path, "r") as file:
    class_names = [line.strip() for line in file.readlines()]

# Define the YAML content with the names dynamically added
data_yaml_content = f"""
train: /home/working/dataset_new/train/images
val: /home/working/dataset_new/val/images 

nc: {len(class_names)}  # Number of classes

names: {class_names}
"""

# Write to the YAML file
yaml_file_path = "/home/working/data.yaml"
with open(yaml_file_path, "w") as file:
    file.write(data_yaml_content)

print("data.yaml file has been created successfully!")


data.yaml file has been created successfully!


In [ ]:
# # Define the YAML content
# data_yaml_content = """
# train: /home/working/dataset_new/train/images
# val: /home/working/dataset_new/val/images 

# nc: 23  # Number of classes

# names: [
#     "MBL", "PM", "Er", "M", "LLC", "EO", "PNN", "LF", "BA", "MM", "LZMG", 
#     "Lysee", "LyB", "SS", "LM", "LY", "LH_lyAct", "B", "MoB", "LGL", "LAM3", 
#     "MO", "Thromb"
# ]
# """

# # Write to the YAML file
# with open("/home/working/data.yaml", "w") as file:
#     file.write(data_yaml_content)




In [ ]:
from ultralytics import YOLO

# Load the YOLOv8 model (pre-trained)
model = YOLO("yolov8s.pt")  # Use 'yolov8n.pt' (nano) for speed or 'yolov8s.pt' (small) for better accuracys

# Train the model
model.train(
    data="/home/working/data.yaml",       # Path to your dataset YAML file
    epochs=20,              # Number of epochs
    batch=64,               # Batch size
    imgsz=640,              # Image size
    name="yolo8_class_bbox" # Experiment name
)


In [ ]:
# # Run inference on a single image or a directory of images
# results = model.predict(source='/home/working/dataset_split/val/images', save=True)


In [25]:
# for result in results:
#     boxes = result.boxes  # Bounding boxes
#     for box in boxes:
#         cls = box.cls  # Class index
#         conf = box.conf  # Confidence score
#         xyxy = box.xyxy  # Bounding box coordinates (x_min, y_min, x_max, y_max)
#         print(f'Class: {cls}, Confidence: {conf}, BBox Coordinates: {xyxy}')
